ALL INTEGRATED

## final code

In [3]:
import pandas as pd
import re
from fuzzywuzzy import process as fuzzy_process
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from datetime import datetime, timedelta
from dateutil import parser
from nltk.stem import WordNetLemmatizer

# Load the Excel files
distance_matrix_path = 'distance_mat_temp (1).xlsx'
events_path = 'Events (1).xlsx'

# Read the distance matrix and events data
distance_matrix = pd.read_excel(distance_matrix_path, index_col=0)
df = pd.read_excel(events_path)

# Extract month from the date column and add it to a new 'Month' column
df['Month'] = pd.to_datetime(df['Start Date']).dt.strftime('%B')

# Combine 'Primary Category', 'Description', and 'Month' into a new 'Combined' column
df['Combined'] = df['Location'].astype(str) + " : " + df['Primary Category'].astype(str) + " : " + df['Sub Category'].astype(str) + " : " + df['Event Names'].astype(str) + " : " + df['Description'].astype(str) + " : " + df['Month']
df['Location_Combined'] = df['Location'].astype(str) + " : " + df['Description'].astype(str)

lemmatizer = WordNetLemmatizer()

# Function to normalize text (e.g., singular vs. plural)
def normalize_text(text):
    words = text.split()
    normalized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(normalized_words)

# Function to find events within a specified radius
def find_events_within_radius(user_location, radius_km):
    if user_location not in distance_matrix.columns:
        return pd.DataFrame()  # Return an empty DataFrame if location is not found
    
    # Find distances from user location to all event locations
    distances = distance_matrix[user_location]
    
    # Filter events based on the radius
    nearby_locations = distances[distances <= radius_km].index
    nearby_events = df[df['Location'].isin(nearby_locations)]
    
    # Add the distance column to nearby events and sort by distance
    nearby_events = nearby_events.assign(Distance=distances[nearby_events['Location']].values)
    nearby_events = nearby_events.sort_values(by='Distance')
    
    return nearby_events

# Function to correct the query based on known locations
def correct_query(query, exclusion_list):
    spell = SpellChecker()
    
    # Normalize and tokenize the query while keeping punctuations
    tokens = re.findall(r"[\w'-]+|[.,!?;]", query)
    
    # Process tokens to handle multi-word exclusions and corrections
    corrected_tokens = []
    for i, token in enumerate(tokens):
        if not token:  # Check for None or empty tokens
            continue
        combined_token = token
        if i + 1 < len(tokens) and tokens[i + 1] not in ".,!?;":
            combined_token += " " + tokens[i + 1]

        # Find the best match from the exclusion list
        match_score = fuzzy_process.extractOne(combined_token, exclusion_list, score_cutoff=80)
        
        if match_score:
            match, score = match_score
            # If a good match is found, use it and skip the next token if it was part of the match
            corrected_tokens.append(match)
        else:
            # If no match, process normally
            corrected_token = spell.correction(token.lower())
            if token[0].isupper():  # Preserve capitalization
                corrected_token = corrected_token.capitalize()
            corrected_tokens.append(corrected_token)

    # Join tokens, handle spaces around punctuation
    corrected_query = ''.join([' '+i if isinstance(i, str) and not i.startswith("'") and i not in ".,!?;" else i for i in corrected_tokens if isinstance(i, str)]).strip()
    return corrected_query

# Function to get date range from the query
def get_date_range(query):
    today = datetime.today()
    if 'today' in query:
        start_date = end_date = today
    elif 'tomorrow' in query:
        start_date = end_date = today + timedelta(days=1)
    elif 'day after tomorrow' in query:
        start_date = end_date = today + timedelta(days=2)
    elif 'this weekend' in query:
        start_date = today + timedelta((5 - today.weekday()) % 7) 
        end_date = start_date + timedelta(days=1)  
    elif 'next weekend' in query:
        start_date = today + timedelta((12 - today.weekday())) 
        end_date = start_date + timedelta(days=1)  
    elif 'next month' in query:
        first_next_month = today.replace(day=1) + timedelta(days=31)
        start_date = first_next_month.replace(day=1)
        end_date = start_date.replace(month=start_date.month % 12 + 1, day=1) - timedelta(days=1)
    else:
        start_date = min(df['Start Date'])
        end_date = max(df['End Date'])
    return start_date, end_date

# Function to parse date from query
def parse_date_from_query(query):
    try:
        date = parser.parse(query, fuzzy=True)
        return date
    except ValueError:
        return None

# Function to search events by date
def search_events_by_date(query, df):
    query = query.lower()
    date = parse_date_from_query(query)
    if not date:
        return pd.DataFrame()
    
    day = date.day
    month = date.month
    filtered_events = df[(df['Start Date'].dt.day <= day) & 
                         (df['End Date'].dt.day >= day) & 
                         (df['Start Date'].dt.month <= month) & 
                         (df['End Date'].dt.month >= month)]
    return filtered_events

# Function to process user query and return filtered events
def process_user_query(query, user_location):
    radius_km = 10000
    location_related = False
    
    patterns = [
        r'events within (\d+)\s*km of (\w+)',
        r'within (\d+)\s*km of (\w+)',
        r'events near (\w+)',
        r'events around (\w+)',
        r'near (\w+)',
        r'around (\w+)',
        r'events within (\d+)\s*km',
        r'within (\d+)\s*km',
        r'(\d+)\s*km radius',
        r'near me',
        r'around me'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, query, re.IGNORECASE)
        if match:
            location_related = True
            if 'me' in query.lower():
                if 'within' in query or 'radius' in query:
                    radius_km = int(match.group(1))
            else:
                if len(match.groups()) == 2:
                    radius_km = int(match.group(1))
                    user_location = match.group(2)
                elif len(match.groups()) == 1:
                    if 'km' in query:
                        radius_km = int(match.group(1))
                    else:
                        user_location = match.group(1)
            break
    
    if location_related:
        nearby_events = find_events_within_radius(user_location, radius_km)
    else:
        nearby_events = pd.DataFrame()

    known_locations = df['Location'].dropna().unique().tolist()
    known_locations.extend(df['Month'].unique().tolist())
    corrected_query = correct_query(query, known_locations)
    
    if re.search(r'\d', corrected_query) and not re.search(r'\b(km|within)\b', corrected_query):
        return search_events_by_date(corrected_query, df)
    
    event_categories = df['Primary Category'].dropna().unique().tolist()
    sub_categories = df['Sub Category'].dropna().unique().tolist()
    
    matched_categories = []
    matched_sub_categories = []
    
    normalized_corrected_query = normalize_text(corrected_query)
    
    for category in event_categories:
        match_score = fuzzy_process.extractOne(normalized_corrected_query, [normalize_text(category)], score_cutoff=80)
        if match_score:
            matched_categories.append(match_score[0])
    
    for subcategory in sub_categories:
        match_score = fuzzy_process.extractOne(normalized_corrected_query, [normalize_text(subcategory)], score_cutoff=80)
        if match_score:
            matched_sub_categories.append(match_score[0])
    
    start_date, end_date = get_date_range(corrected_query)
    filtered_df = df[(df['Start Date'] >= start_date.strftime('%Y-%m-%d')) & (df['End Date'] <= end_date.strftime('%Y-%m-%d'))]
    
    month_names = [month.lower() for month in df['Month'].dropna().unique()]
    month_pattern = r'\b(' + '|'.join(month_names) + r')\b'
    month_match = re.search(month_pattern, corrected_query, re.IGNORECASE)
    if month_match:
        month = month_match.group(1)
        if month:
            month = month.capitalize()
            filtered_df = filtered_df[filtered_df['Month'].str.lower() == month.lower()]
    
    if matched_categories or matched_sub_categories:
        filtered_df = filtered_df[(filtered_df['Primary Category'].isin(matched_categories)) | (filtered_df['Sub Category'].isin(matched_sub_categories))]
    
    if nearby_events.empty and (matched_categories or matched_sub_categories):
        nearby_events = df[(df['Primary Category'].isin(matched_categories)) | (df['Sub Category'].isin(matched_sub_categories))]

    combined_df = pd.concat([nearby_events, filtered_df]).drop_duplicates(subset=['Event ID'])

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(combined_df['Combined'].tolist())
    query_vector = vectorizer.transform([corrected_query])

    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()
    combined_df['Cosine Similarity'] = cosine_similarities

    if location_related and 'Distance' in combined_df.columns:
        combined_df = combined_df.sort_values(by=['Distance', 'Cosine Similarity'], ascending=[True, False])
    else:
        combined_df = combined_df.sort_values(by='Cosine Similarity', ascending=False)

    return combined_df

# Function to search events by query if numeric value is found in the query
def search_events_by_query(file_path, user_query):
    events_df = pd.read_excel(file_path)
    events_df['Start Date'] = pd.to_datetime(events_df['Start Date'])
    events_df['End Date'] = pd.to_datetime(events_df['End Date'])
    
    return search_events_by_date(user_query, events_df)

# Interactive part to get user query and display results
user_location = input("Enter your location: ").strip()
user_query = input("Enter your query (e.g., 'events within 5km of Aundh' or 'events near me'): ").strip().lower()

if re.search(r'\b(km|within)\b', user_query):
    result = process_user_query(user_query, user_location)
elif any(char.isdigit() for char in user_query):
    result = search_events_by_query(events_path, user_query)
else:
    result = process_user_query(user_query, user_location)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Events", dataframe=result)

result

C:\Users\pratik\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Enter your location:  Shivajinagar
Enter your query (e.g., 'events within 5km of Aundh' or 'events near me'):  concerts at bavdhan


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Event Type,Prices,Payment Model,...,Start Day,End Day,Start Time,End Time,Duration,Number of Days,Links,Combined,Location_Combined,Cosine Similarity
2,EVT0003,Ajay Atul Live,Let's enjoy the magic of Ajay-Atul’s music lik...,Music and Entertainment,NaN,Concert,Bavdhan,In-Person,1500,Paid,...,Saturday,Saturday,19:00:00,21:00:00,02:00:00,1,https://in.bookmyshow.com/events/ajay-atul-liv...,Bavdhan : Music and Entertainment : Concert : ...,Bavdhan : Let's enjoy the magic of Ajay-Atul’s...,0.184629
16,EVT0017,Vishal Mishra India Tour,VISHAL MISHRA “PEHLE BHI MAIN” INDIA TOUR 🇮🇳Vi...,Music and Entertainment,NaN,Concert,Kharadi,Virtual,499,Paid,...,Saturday,Saturday,18:00:00,18:50:00,00:50:00,1,NaN,Kharadi : Music and Entertainment : Concert : ...,Kharadi : VISHAL MISHRA “PEHLE BHI MAIN” INDIA...,0.000000
50,EVT0052,Friday Diva Night - DJ Veronica,CELEBRATE FRIDAY DIVA NIGHT WITH CHEERS TO YOU...,Music and Entertainment,NaN,Concert,Koregaon Park,In-Person,2500,Paid,...,Monday,Monday,18:00:00,19:30:00,01:30:00,1,https://allevents.in/pune/friday-diva-night-dj...,Koregaon Park : Music and Entertainment : Conc...,Koregaon Park : CELEBRATE FRIDAY DIVA NIGHT WI...,0.000000
87,EVT0090,Mehfil,"Join us at ""Mehfil,"" a vibrant community event...",Music and Entertainment,NaN,Concert,Lullanagar,In-Person,300,Paid,...,Sunday,Sunday,19:00:00,20:40:00,01:40:00,1,https://in.bookmyshow.com/events/mehfil/ET0036...,Lullanagar : Music and Entertainment : Concert...,"Lullanagar : Join us at ""Mehfil,"" a vibrant co...",0.000000
104,EVT0107,Vishal and Sheykhar Live,"Vishal-Shekhar, formed in 1999, is an Indian m...",Music and Entertainment,NaN,Concert,Deccan,In-Person,499,Paid,...,Saturday,Saturday,17:00:00,18:20:00,01:20:00,1,https://in.bookmyshow.com/events/urbane-shows-...,Deccan : Music and Entertainment : Concert : V...,"Deccan : Vishal-Shekhar, formed in 1999, is an...",0.000000
110,EVT0113,Time Throttle,Time Throttle is a progressive rock band that ...,Music and Entertainment,NaN,Concert,Camp,Virtual,299,Paid,...,Sunday,Sunday,21:00:00,23:10:00,02:10:00,1,NaN,Camp : Music and Entertainment : Concert : Tim...,Camp : Time Throttle is a progressive rock ban...,0.000000
111,EVT0114,Yamak Wagare (Maifil),चला तर मग तरुणाईने सजलेल्या जुन्या नवीन गाण्या...,Music and Entertainment,NaN,Concert,Deccan,In-Person,199,Paid,...,Wednesday,Wednesday,19:00:00,21:10:00,02:10:00,1,NaN,Deccan : Music and Entertainment : Concert : Y...,Deccan : चला तर मग तरुणाईने सजलेल्या जुन्या नव...,0.000000
112,EVT0115,Music at the Rotunda,NaN,Music and Entertainment,NaN,Concert,Camp,Virtual,399,Paid,...,Thursday,Thursday,18:00:00,19:10:00,01:10:00,1,NaN,Camp : Music and Entertainment : Concert : Mus...,Camp : nan,0.000000
158,EVT0161,Shah Jo Risalo by Ashwin 'Afraad',Shah Jo Risalo - an Hour-long mehfil recounts ...,Music and Entertainment,NaN,Concert,Lullanagar,In-Person,300,Paid,...,Sunday,Sunday,07:00:00,08:20:00,01:20:00,1,"Shah Jo Risalo by Ashwin Afraad Tickets, Raah ...",Lullanagar : Music and Entertainment : Concert...,Lullanagar : Shah Jo Risalo - an Hour-long meh...,0.000000
163,EVT0166,Shayarana,"It’s a Shero – Shayari, Poetic and Old Hindi m...",Music and Entertainment,NaN,Concert,Kothrud,In-Person,300,Paid,...,Friday,Friday,21:30:00,23:50:00,02:20:00,1,https://in.bookmyshow.com/pune/events/shayaran...,Kothrud : Music and Entertainment : Concert : ...,"Kothrud : It’s a Shero – Shayari, Poetic and O...",0.000000


# Event Query Processing and Filtering

This Python notebook demonstrates the processing of user queries to find relevant events from a dataset based on various criteria such as location, distance, and date. The following sections explain the key components and functionality of the code.

## 1. Loading and Preparing the Data

- **Loading the Data**: 
  Two Excel files are loaded: one containing a distance matrix (`distance_mat_temp (1).xlsx`) and another containing event data (`Events (1).xlsx`).

- **Preprocessing the Data**: 
  - The `Month` column is created by extracting the month from the `Start Date` of each event.
  - A `Combined` column is added, which concatenates event details like location, category, subcategory, and description. This is used later for easy querying.
  - The `Location_Combined` column contains only location and description data.

## 2. Text Normalization and Lemmatization

- **Text Normalization**: 
  The `normalize_text` function uses lemmatization to convert words to their base form. This ensures that plural forms and other variations are standardized (e.g., "running" becomes "run").

```python
def normalize_text(text):
    words = text.split()
    normalized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(normalized_words)


In [6]:
import pandas as pd
import re
from fuzzywuzzy import process as fuzzy_process
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from datetime import datetime, timedelta
from dateutil import parser
from nltk.stem import WordNetLemmatizer

# Load the Excel files
distance_matrix_path = 'distance_mat_temp (1).xlsx'
events_path = 'Events (1).xlsx'

# Read the distance matrix and events data
distance_matrix = pd.read_excel(distance_matrix_path, index_col=0)
df = pd.read_excel(events_path)

# Extract month from the date column and add it to a new 'Month' column
df['Month'] = pd.to_datetime(df['Start Date']).dt.strftime('%B')

# Combine 'Primary Category', 'Description', and 'Month' into a new 'Combined' column
df['Combined'] = df['Location'].astype(str) + " : " + df['Primary Category'].astype(str) + " : " + df['Sub Category'].astype(str) + " : " + df['Event Names'].astype(str) + " : " + df['Description'].astype(str) + " : " + df['Month']
df['Location_Combined'] = df['Location'].astype(str) + " : " + df['Description'].astype(str)

lemmatizer = WordNetLemmatizer()

# Function to normalize text (e.g., singular vs. plural)
def normalize_text(text):
    words = text.split()
    normalized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(normalized_words)

# Function to find events within a specified radius
def find_events_within_radius(user_location, radius_km):
    if user_location not in distance_matrix.columns:
        return pd.DataFrame()  # Return an empty DataFrame if location is not found
    
    # Find distances from user location to all event locations
    distances = distance_matrix[user_location]
    
    # Filter events based on the radius
    nearby_locations = distances[distances <= radius_km].index
    nearby_events = df[df['Location'].isin(nearby_locations)]
    
    # Add the distance column to nearby events and sort by distance
    nearby_events = nearby_events.assign(Distance=distances[nearby_events['Location']].values)
    nearby_events = nearby_events.sort_values(by='Distance')
    
    return nearby_events

# Function to correct the query based on known locations
def correct_query(query, exclusion_list):
    spell = SpellChecker()

    # Normalize and tokenize the query while keeping punctuations
    tokens = re.findall(r"[\w'-]+|[.,!?;]", query)

    # Process tokens to handle multi-word exclusions and corrections
    corrected_tokens = []
    i = 0
    while i < len(tokens):
        token = tokens[i]
        combined_token = token
        if i + 1 < len(tokens) and tokens[i + 1] not in ".,!?;":
            combined_token += " " + tokens[i + 1]
        
        # Find the best match from the exclusion list
        match_score = fuzzy_process.extractOne(combined_token, exclusion_list, score_cutoff=80)
        if match_score:
            match, score = match_score
            corrected_tokens.append(match)
            i += 2  # Skip the next token as it was part of the match
        else:
            # If no match, process normally
            corrected_token = spell.correction(token.lower())
            if token[0].isupper():  # Preserve capitalization
                corrected_token = corrected_token.capitalize()
            corrected_tokens.append(corrected_token)
            i += 1

    # Join tokens, handle spaces around punctuation
    corrected_query = ''.join([' '+i if isinstance(i, str) and not i.startswith("'") and i not in ".,!?;" else i for i in corrected_tokens if isinstance(i, str)]).strip()
    return corrected_query

# Function to get date range from the query
def get_date_range(query):
    today = datetime.today()
    if 'today' in query:
        start_date = end_date = today
    elif 'tomorrow' in query:
        start_date = end_date = today + timedelta(days=1)
    elif 'day after tomorrow' in query:
        start_date = end_date = today + timedelta(days=2)
    elif 'this weekend' in query:
        start_date = today + timedelta((5 - today.weekday()) % 7) 
        end_date = start_date + timedelta(days=1)  
    elif 'next weekend' in query:
        start_date = today + timedelta((12 - today.weekday())) 
        end_date = start_date + timedelta(days=1)  
    elif 'next month' in query:
        first_next_month = today.replace(day=1) + timedelta(days=31)
        start_date = first_next_month.replace(day=1)
        end_date = start_date.replace(month=start_date.month % 12 + 1, day=1) - timedelta(days=1)
    else:
        start_date = min(df['Start Date'])
        end_date = max(df['End Date'])
    return start_date, end_date

# Function to parse date from query
def parse_date_from_query(query):
    try:
        date = parser.parse(query, fuzzy=True)
        return date
    except ValueError:
        return None

# Function to search events by date
def search_events_by_date(query, df):
    query = query.lower()
    date = parse_date_from_query(query)
    if not date:
        return pd.DataFrame()
    
    day = date.day
    month = date.month
    filtered_events = df[(df['Start Date'].dt.day <= day) & 
                         (df['End Date'].dt.day >= day) & 
                         (df['Start Date'].dt.month <= month) & 
                         (df['End Date'].dt.month >= month)]
    return filtered_events

# Function to process user query and return filtered events
def process_user_query(query, user_location):
    radius_km = 10000
    location_related = False
    
    patterns = [
        r'events within (\d+)\s*km of (\w+)',
        r'within (\d+)\s*km of (\w+)',
        r'events near (\w+)',
        r'events around (\w+)',
        r'near (\w+)',
        r'around (\w+)',
        r'events within (\d+)\s*km',
        r'within (\d+)\s*km',
        r'(\d+)\s*km radius',
        r'near me',
        r'around me'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, query, re.IGNORECASE)
        if match:
            location_related = True
            if 'me' in query.lower():
                if 'within' in query or 'radius' in query:
                    radius_km = int(match.group(1))
            else:
                if len(match.groups()) == 2:
                    radius_km = int(match.group(1))
                    user_location = match.group(2)
                elif len(match.groups()) == 1:
                    if 'km' in query:
                        radius_km = int(match.group(1))
                    else:
                        user_location = match.group(1)
            break
    
    if location_related:
        nearby_events = find_events_within_radius(user_location, radius_km)
    else:
        nearby_events = pd.DataFrame()

    known_locations = df['Location'].dropna().unique().tolist()
    known_locations.extend(df['Month'].unique().tolist())
    corrected_query = correct_query(query, known_locations)
    
    if re.search(r'\d', corrected_query) and not re.search(r'\b(km|within)\b', corrected_query):
        return search_events_by_date(corrected_query, df)
    
    event_categories = df['Primary Category'].dropna().unique().tolist()
    sub_categories = df['Sub Category'].dropna().unique().tolist()
    
    matched_categories = []
    matched_sub_categories = []
    
    normalized_corrected_query = normalize_text(corrected_query)
    
    for category in event_categories:
        match_score = fuzzy_process.extractOne(normalized_corrected_query, [normalize_text(category)], score_cutoff=80)
        if match_score:
            matched_categories.append(match_score[0])
    
    for subcategory in sub_categories:
        match_score = fuzzy_process.extractOne(normalized_corrected_query, [normalize_text(subcategory)], score_cutoff=80)
        if match_score:
            matched_sub_categories.append(match_score[0])
    
    start_date, end_date = get_date_range(corrected_query)
    filtered_df = df[(df['Start Date'] >= start_date.strftime('%Y-%m-%d')) & (df['End Date'] <= end_date.strftime('%Y-%m-%d'))]
    
    month_names = [month.lower() for month in df['Month'].dropna().unique()]
    month_pattern = r'\b(' + '|'.join(month_names) + r')\b'
    month_match = re.search(month_pattern, corrected_query, re.IGNORECASE)
    if month_match:
        month = month_match.group(1)
        if month:
            month = month.capitalize()
            filtered_df = filtered_df[filtered_df['Month'].str.lower() == month.lower()]
    
    if matched_categories or matched_sub_categories:
        filtered_df = filtered_df[(filtered_df['Primary Category'].isin(matched_categories)) | (filtered_df['Sub Category'].isin(matched_sub_categories))]
    
    if nearby_events.empty and (matched_categories or matched_sub_categories):
        nearby_events = df[(df['Primary Category'].isin(matched_categories)) | (df['Sub Category'].isin(matched_sub_categories))]

    combined_df = pd.concat([nearby_events, filtered_df]).drop_duplicates(subset=['Event ID'])

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(combined_df['Combined'].tolist())
    query_vector = vectorizer.transform([corrected_query])

    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()
    combined_df['Cosine Similarity'] = cosine_similarities

    if location_related and 'Distance' in combined_df.columns:
        combined_df = combined_df.sort_values(by=['Distance', 'Cosine Similarity'], ascending=[True, False])
    else:
        combined_df = combined_df.sort_values(by='Cosine Similarity', ascending=False)

    return combined_df

# Function to search events by query if numeric value is found in the query
def search_events_by_query(file_path, user_query):
    events_df = pd.read_excel(file_path)
    events_df['Start Date'] = pd.to_datetime(events_df['Start Date'])
    events_df['End Date'] = pd.to_datetime(events_df['End Date'])
    
    return search_events_by_date(user_query, events_df)

# Interactive part to get user query and display results
user_location = input("Enter your location: ").strip()
user_query = input("Enter your query (e.g., 'events within 5km of Aundh' or 'events near me'): ").strip().lower()

if re.search(r'\b(km|within)\b', user_query):
    result = process_user_query(user_query, user_location)
elif any(char.isdigit() for char in user_query):
    result = search_events_by_query(events_path, user_query)
else:
    result = process_user_query(user_query, user_location)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Events", dataframe=result)

result

C:\Users\pratik\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Enter your location:  f
Enter your query (e.g., 'events within 5km of Aundh' or 'events near me'):  ajay atul


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Event Type,Prices,Payment Model,...,Start Day,End Day,Start Time,End Time,Duration,Number of Days,Links,Combined,Location_Combined,Cosine Similarity
175,EVT0178,Obstacle Course - Pune,Bounce through obstacles chilling at the refre...,Sport,Fitness,Marathon,Pune,Virtual,0,Free,...,Sunday,Sunday,07:00:00,07:40:00,00:40:00,1,The Great Inflate Run - Pune - Indias First In...,Pune : Sport : Marathon : Obstacle Course - Pu...,Pune : Bounce through obstacles chilling at th...,0.179590
197,EVT0200,THE LIT BUG FEST 2024 SEASON 6\n,Experience literature from all facets,Art and Culture,Music and Entertainment,NaN,Baner,In-Person,700,Paid,...,Saturday,Saturday,10:00:00,12:10:00,02:10:00,1,https://in.bookmyshow.com/activities/the-lit-b...,Baner : Art and Culture : nan : THE LIT BUG FE...,Baner : Experience literature from all facets,0.125555
174,EVT0177,All India Quick ChessChampionship by Malsar.in,Chess event for every category in a competetiv...,Sport,NaN,Chess,Pune,Virtual,249,Paid,...,Saturday,Saturday,19:30:00,20:00:00,00:30:00,1,https://in.bookmyshow.com/pune/sports/all-indi...,Pune : Sport : Chess : All India Quick ChessCh...,Pune : Chess event for every category in a com...,0.101388
19,EVT0020,Marathon,"All Timed Run Participants will get Bib Medal,...",Fitness,NaN,Marathon,Aundh,Virtual,0,Free,...,Thursday,Thursday,08:00:00,09:50:00,01:50:00,1,NaN,Aundh : Fitness : Marathon : Marathon : All Ti...,Aundh : All Timed Run Participants will get Bi...,0.093195
37,EVT0038,Kalaa Spandan Art Fair,A leading International platform for contempor...,Art and Culture,NaN,Exhibition,Karve Nagar,In-Person,200,Paid,...,Thursday,Thursday,07:00:00,08:50:00,01:50:00,1,https://allevents.in/pune/kalaa-spandan-art-fa...,Karve Nagar : Art and Culture : Exhibition : K...,Karve Nagar : A leading International platform...,0.089348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,EVT0075,Healing and Self Discovery,Experience a transformative journey of Healing...,Fitness,NaN,Meditation and Mindfulness,Koregaon Park,In-Person,999,Paid,...,Saturday,Saturday,09:30:00,11:30:00,02:00:00,1,https://allevents.in/pune/shift-1-day-self-dis...,Koregaon Park : Fitness : Meditation and Mindf...,Koregaon Park : Experience a transformative jo...,0.000000
73,EVT0076,Soft Pastel Atelier Workshop,Discover the enchanting allure of soft pastels...,Art and Culture,NaN,Drawing and Painting\n,Ghorpadi,In-Person,1500,Paid,...,Sunday,Sunday,14:30:00,15:30:00,01:00:00,1,NaN,Ghorpadi : Art and Culture : Drawing and Paint...,Ghorpadi : Discover the enchanting allure of s...,0.000000
74,EVT0077,Stargazing Party,Basic Sessions On Star Gazing (Identifying con...,Music and Entertainment,Travel,Tour,Yerawada,In-Person,2300,Paid,...,Saturday,Saturday,17:00:00,17:50:00,00:50:00,1,https://allevents.in/pune/stargazing-party-and...,Yerawada : Music and Entertainment : Tour : St...,Yerawada : Basic Sessions On Star Gazing (Iden...,0.000000
75,EVT0078,Come and Perform - Open Mic,“Rock the Stage!” is an exhilarating Open Mic ...,Music and Entertainment,NaN,Comedy,Lullanagar,In-Person,250,Paid,...,Saturday,Saturday,18:00:00,19:10:00,01:10:00,1,NaN,Lullanagar : Music and Entertainment : Comedy ...,Lullanagar : “Rock the Stage!” is an exhilarat...,0.000000
